In [1]:
!pip install geemap
!pip install eerepr
!pip install geopandas
import os,tarfile
import geemap
import subprocess
import ee
import eerepr
import ipywidgets as widgets
from bqplot import pyplot as plt
from ipyleaflet import WidgetControl
import numpy as np
import pandas as pd
import folium
import copy
import math
from tqdm import trange,tqdm
from functools import partial
import geopandas as gpd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 83.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.8 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none

In [2]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=4nwNj_d6Wu3KUOiQweYSFt2BIAilJJBH21F1Xc-oXgw&tc=JbVboU7neTgve49VQ09D6Qnw8ICEp-jgdkFb-2T5mOY&cc=mkFphunRBNprXK81nqUUfThUzO0dRxHdRu5-35L3Sgo

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AbUR2VPr1u5Tsu9abTltZT1DiVDgZTRNNq8hYM-q8YA41JN2838gP10u21I

Successfully saved authorization token.


In [3]:
Glacial_lake_2015A = ee.FeatureCollection('projects/ee-mrwurenzhe/assets/Glacial_lake/Wu_Asia_Southest_GL_wgs84').filter(ee.Filter.gte('GL_Area', 0.1))  
Geo_ext = lambda feature: feature.set({
    'Geo': feature.geometry(),
    'Centroid': feature.geometry().centroid(),
    'Rectangle': feature.geometry().bounds()
})
Centrid_set = lambda feature: feature.setGeometry(feature.geometry().centroid())
Rectangle_set = lambda feature: feature.setGeometry(feature.geometry().bounds())

Glacial_lake_2015C = Glacial_lake_2015A.map(Geo_ext).map(Centrid_set)  
Glacial_lake_2015R = Glacial_lake_2015A.map(Rectangle_set)       

Glacial_lake_2015A_GeoList = ee.List(Glacial_lake_2015C.reduceColumns(ee.Reducer.toList(), ['Geo']).get('list'))
Glacial_lake_2015C_CentriodList = ee.List(Glacial_lake_2015C.reduceColumns(ee.Reducer.toList(),['Centroid']).get('list'))
Glacial_lake_2015R_RectangleList = ee.List(Glacial_lake_2015C.reduceColumns(ee.Reducer.toList(),['Rectangle']).get('list'))
Num_list = Glacial_lake_2015C_CentriodList.size().getInfo()

In [4]:
# 预定义函数
def make_dir(path):
    isExists = os.path.exists(path)
    # 判断结果
    if not isExists:
        os.makedirs(path)
        print(path + ' 创建成功')
    return path

def cut_geometry(geometry,block_size:float=0.05):
  '''
  block_size 定义方块大小(度)

  '''
  # 计算边界
  bounds = ee.List(ee.List(geometry.bounds().coordinates()).get(0))

  # 计算geometry的宽度和高度
  width = ee.Number(ee.List(bounds.get(2)).get(0)).subtract(ee.Number(ee.List(bounds.get(0)).get(0)))
  height = ee.Number(ee.List(bounds.get(2)).get(1)).subtract(ee.Number(ee.List(bounds.get(0)).get(1)))

  # 计算行和列的数量
  num_rows = height.divide(block_size).ceil()
  num_cols = width.divide(block_size).ceil()

  # 定义一个函数，用于生成512x512的方块
  def create_blocks(row, col):
      x_min = ee.Number(ee.List(bounds.get(0)).get(0)).add(col.multiply(block_size))
      y_min = ee.Number(ee.List(bounds.get(0)).get(1)).add(row.multiply(block_size))
      x_max = x_min.add(block_size)
      y_max = y_min.add(block_size)
      return ee.Geometry.Rectangle([x_min, y_min, x_max, y_max])

  # 生成方块列表
  block_list = []
  for row in range(num_rows.getInfo()):
      for col in range(num_cols.getInfo()):
          block = create_blocks(ee.Number(row), ee.Number(col))
          block_list.append(block)
  return block_list

def clip_AOI(col, AOI):
    return col.clip(AOI)

def time_difference(col, middle_date):
    time_difference = middle_date.difference(
        ee.Date(col.get('system:time_start')), 'days').abs()
    return col.set({'time_difference': time_difference})

def rm_nodata(col, AOI):
    # 将遮罩外的元素置-99,默认的遮罩为Nodata区域，并统计Nodata的数量
    allNone_num = col.select('VV_sigma0').unmask(-99).eq(-99).reduceRegion(
        **{
            'geometry': AOI,
            'reducer': ee.Reducer.sum(),
            'scale': 10,
            'maxPixels': 1e12,
            'bestEffort': True,
        }).get('VV_sigma0')
    return col.set({'numNodata': allNone_num})

def load_image_collection(aoi, start_date, end_date, middle_date,
                         dem=None,model=None,buffer=0,Filter=None,FilterSize=30):
    '''
    s1数据加载
    '''
    s1_col = (ee.ImageCollection("COPERNICUS/S1_GRD")
              .filter(ee.Filter.eq('instrumentMode', 'IW'))
                .filterBounds(aoi)
                .filterDate(start_date, end_date))
    # 图像滤波，可选
    if Filter:
        print('Begin Filter ...')
        if Filter == 'leesigma':
            s1_col = s1_col.map(leesigma(FilterSize))
        elif Filter == 'RefinedLee':
            s1_col = s1_col.map(RefinedLee)
        elif Filter == 'gammamap':
            s1_col = s1_col.map(gammamap(FilterSize))
        elif Filter == 'boxcar':
            s1_col = s1_col.map(boxcar(FilterSize))
        else:
          print('Wrong Filter')
    else:
        print('Without Filter')
        
    # 地形矫正，可选
    if dem and model:
        print('Begin Slop Correction ...')
        s1_col = slope_correction(s1_col, dem, model, buffer=buffer)
    else:
        Rename = lambda image:image.rename(['VV_sigma0', 'VH_sigma0','incAngle'])
        s1_col = s1_col.map(Rename)
        print('Without Slop Correction')
        
    # 裁剪并计算空洞数量
    s1_col = s1_col.map(partial(clip_AOI, AOI=aoi))
    s1_col = s1_col.map(partial(rm_nodata, AOI=aoi))

    # 计算与目标日期的差距
    s1_col = s1_col.map(partial(time_difference, middle_date=middle_date))
    
    # 删除角度波段select
    if model:
      pass
    else:
      s1_col = s1_col.map(lambda img: img.select(['VV_sigma0', 'VH_sigma0']))
      
    # 分离升降轨
    s1_descending = s1_col.filter(ee.Filter.eq(
        'orbitProperties_pass', 'DESCENDING'))
    s1_ascending = s1_col.filter(ee.Filter.eq(
        'orbitProperties_pass', 'ASCENDING'))
    
    # 如果所有图像都包含空洞则返回一张合成的图像，还需要重投影，否则计算会出错
    if s1_descending.filter(ee.Filter.eq('numNodata', 0)).size().getInfo() == 0:
      s1_descending = s1_descending.median().reproject(s1_descending.first().projection().getInfo()['crs'], None, 10).clip(aoi)
    else:
      s1_descending = s1_descending.filter(ee.Filter.eq('numNodata', 0))

    if s1_ascending.filter(ee.Filter.eq('numNodata', 0)).size().getInfo() == 0:
      s1_ascending = s1_ascending.median().reproject(s1_ascending.first().projection().getInfo()['crs'], None, 10).clip(aoi)
    else:
      s1_ascending = s1_ascending.filter(ee.Filter.eq('numNodata', 0))

    return s1_ascending, s1_descending

def Geemap_export(filename,collection=False,image=False,region=None,scale=10):
    if collection:
        # 这里导出时候使用region设置AOI，否则可能因为坐标系问题(未确定)，出现黑边问题
        geemap.ee_export_image_collection(collection,
                        out_dir=os.path.dirname(out_dir),
                        format = "ZIPPED_GEO_TIFF",region=region,scale=scale)
        print('collection save right')
    elif image:     
        geemap.ee_export_image(image,
                    filename=filename,
                    scale=scale, region=region, file_per_band=False,timeout=1500)
        print('image save right')
    else:
        print('Erro:collection && image must have one False')

In [5]:
CLOUD_FILTER = 60                  # 过滤s2 大于指定云量的数据
CLD_PRB_THRESH = 15                # s2cloudless 概率值阈值[0-100],原实验是50
NIR_DRK_THRESH = 0.15              # 非水暗像素判断阈值
CLD_PRJ_DIST = 1                   # 根据 CLD_PRJ_DIST 输入指定的距离从云中投射阴影
BUFFER = 50                        # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input

def add_cloud_bands(img):
    """Define a function to add the s2cloudless probability layer
    and derived cloud mask as bands to an S2 SR image input."""
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):
    """Define a function to add dark pixels, 
    cloud projection, and identified shadows as bands to an S2 SR image input.
    Note that the image input needs to be the result of the above add_cloud_bands function
    because it relies on knowing which pixels are considered cloudy ('clouds' band)."""
    # 从 SCL 波段识别水像素, 仅适用于L2A，采用L1C计算MNDWI
    not_water = img.select('SCL').neq(6)

    # 识别非水的暗 NIR 像素(潜在的云阴影像素)。.
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # 确定云投射云影的方向(假设是 UTM 投影)。
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # 根据 CLD_PRJ_DIST 输入指定的距离从云中投射阴影
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')
    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    """
    添加cloud和shadow mask
    """
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()
    # Subset reflectance bands and update their masks, return the result.
    return img.select(['B.*','clouds','dark_pixels','shadows','cloudmask']).updateMask(not_cld_shdw)

def merge_s2_collection(aoi, start_date, end_date):
    """筛选S2图像以及S2_cloud图像，并将两个collection连接"""
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)).map(partial(clip_AOI,AOI=aoi)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date).map(partial(clip_AOI,AOI=aoi)))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    # 固定用法，将两个collection通过属性值连接起来，s2cloudless整体作为一个属性写入
    s2_sr_cld_col = ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'})
    }))
    
    s2_sr_cld_col_disp = s2_sr_cld_col.map(add_cld_shdw_mask)
    # 使用clip赋予geometry
    s2_sr_median = s2_sr_cld_col_disp.map(apply_cld_shdw_mask).median().clip(aoi).int16()
    return s2_sr_median

In [7]:
# 选择一个时间点的影像，注意SAR和光学影像的时间点已经不一样了
# 需要放宽一些时间限制，否则可能会因为天气影响、卫星维护等问题出现数据缺失
START_DATE  = ee.Date('2019-07-01')
END_DATE   = ee.Date('2019-08-30')
TIME_LEN   = END_DATE.difference(START_DATE, 'days').abs()
MIDDLE_DATE  = START_DATE.advance(TIME_LEN.divide(ee.Number(2)).int(),'days')
dem = ee.Image('USGS/SRTMGL1_003') # paths to dem 
models = ['volume', 'surface', None]     # 地形矫正模型

# 选择一个点做实验
i=3 #394(面积大) #53 #11(影像都用nodata) #149(坐标系问题)

AOI_point = ee.Feature.geometry(Glacial_lake_2015C_CentriodList.get(i))
AOI_area = ee.Feature.geometry(Glacial_lake_2015A_GeoList.get(i))
AOI = ee.Feature.geometry(Glacial_lake_2015R_RectangleList.get(i))

AOI_buffer = AOI.buffer(distance=2000)
AOI_buffer = AOI_buffer.bounds()
# 在这个点采用buffer裁剪文件，展示SAR，去除了包含空值的影像
s1_ascending,s1_descending = load_image_collection(AOI_buffer,START_DATE,END_DATE,MIDDLE_DATE,
                          dem=dem,model=models[2],Filter=False,FilterSize=30)

# 全包含空值条件下，将图像集合合并为一张图像进行处理
if type(s1_ascending) == ee.imagecollection.ImageCollection:
  s1_single_a = s1_ascending.filter(ee.Filter.eq('time_difference', 
          s1_ascending.aggregate_min('time_difference'))).first().set({'synthesis': 0})
else:
  s1_single_a = s1_ascending.set({'synthesis': 1})
if type(s1_descending) == ee.imagecollection.ImageCollection:
  s1_single_d = s1_descending.filter(ee.Filter.eq('time_difference', 
          s1_descending.aggregate_min('time_difference'))).first().set({'synthesis': 0})
else:
  s1_single_d = s1_descending.set({'synthesis': 1})

# 加入条件空值，保证在空值情况下图像能够正常相加
condition = s1_single_d.mask().clip(AOI_buffer)
s1_unit_mean = s1_single_a.where(condition, s1_single_a.add(s1_single_d).divide(2)) #转为均值
s1_unit_max = s1_single_a.where(condition, s1_single_a.max(s1_single_d))
# s1_unit_add = s1_single_a.where(condition, s1_single_a.add(s1_single_d))

s1_unit_all = s1_unit_mean.addBands(s1_unit_max)

# 采用S2cloudless生产一张合成的无云图像
s2_sr_median = merge_s2_collection(AOI_buffer,ee.Date('2019-06-01'),ee.Date('2019-09-30'))

# 考虑在升降轨都表现为阴影的条件下(升降轨的夹角小，不足以满足在所有条件下阴影去除)

Without Filter
Without Slop Correction


In [9]:
Map = geemap.Map(basemap='HYBRID') # 
Map.centerObject(AOI_point, zoom=15)
# 原始图像
Map.addLayer(s2_sr_median,{'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2000, 'gamma': 2.0},'s2_sr_median')
Map

Map(center=[27.52144300344888, 92.03452315692468], controls=(WidgetControl(options=['position', 'transparent_b…

## 冰湖多视图

In [ ]:
geemap.linked_maps(
    rows=2,
    cols=2,
    height="400px",
    center=AOI_point.getInfo()['coordinates'][::-1],
    zoom=14,
    ee_objects=[s1_single_a.select('VV_sigma0'),
          s1_single_d.select('VV_sigma0'),
          s1_single_a.select('VH_sigma0'),
          s1_single_d.select('VH_sigma0'),],
    vis_params=[{'min':-40,'max':5}],
    # vis_params=[{'band':['VV_sigma0'],'min':-40,'max':5},
    #       {'band':['VH_sigma0','VH_sigma0','VH_sigma0'],'min':-40,'max':5},
    #       {'band':'VV_sigma0_1','min':-40,'max':5},
    #       {'band':'VH_sigma0_1','min':-40,'max':5}],
    labels=['A_VV','D_VV','A_VH','D_VH'],
    label_position="topright",
)

In [79]:
geemap.linked_maps(
    rows=2,
    cols=2,
    height="400px",
    center=AOI_point.getInfo()['coordinates'][::-1],
    zoom=14,
    ee_objects=[s1_unit_all.select('VV_sigma0'),
          s1_unit_all.select('VH_sigma0'),
          s1_unit_all.select('VV_sigma0_1'),
          s1_unit_all.select('VH_sigma0_1')],
    vis_params=[{'min':-40,'max':5}],
    # vis_params=[{'band':['VV_sigma0'],'min':-40,'max':5},
    #       {'band':['VH_sigma0','VH_sigma0','VH_sigma0'],'min':-40,'max':5},
    #       {'band':'VV_sigma0_1','min':-40,'max':5},
    #       {'band':'VH_sigma0_1','min':-40,'max':5}],
    labels=['Mean_VV,','Mean_VH','Max_VV',"Max_VH"],
    label_position="topright",
)

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

## 冰湖快速演化